# Stock Sentiment Analysis Project

What's the goal of the project?
How do we plan to achieve the goal?
What are some technologies used?

# Setup

In [1]:
# import dependencies (for all of the notebook and remove from elsewhere)
import sys
import pandas as pd
from news_finders import content_scraper
from news_finders import newsApi
from yahoofinancials import YahooFinancials
from utils import helper_functions
from utils import sqlite_utils
from datetime import datetime, timedelta
from stock_historical_data import getStockData
from matplotlib import pyplot as plt 


connection_article_database = sqlite_utils.setUpArticleDatabase()

connection_pricing_database = sqlite_utils.setUpPricingDatabase()

## Gathering the Article Data for our Stocks
Utilizing the News API service, we gather articles referencing the Stock Symbols that we are interested in. We t ake the JSON data returned, clean it, and store into our sqlite database for future reference.

In [ ]:
STOCKS = ["WMT", "AMZN", "MRNA"]

get_articles_from_this_date = helper_functions.getMaxDateForStockArticles()

articles = []

articles = newsApi.getArticlesForMultipleStocks(STOCKS, get_articles_from_this_date)

sqlite_utils.insertStockArticles(connection_article_database, articles)

print(articles)

In [ ]:
# Test to make sure that articles have been stored in the database
print(sqlite_utils._findStockArticlesForSymbol(connection_article_database, "WMT")[0])

## Gathering the Historical Price Data for our Stocks
Utilizing the Yahoo Financials API, we gather stock finance data per each company that we are interested in and store that data in a Stock Pricing Database. 

In [ ]:
getStockData.userSearchAndStoreStockPricingLoop(connection_pricing_database)

### Pulling Data from stockSentiment database

In [ ]:
dataFrame = pd.read_sql_query('SELECT * FROM stockArticles WHERE date >= "2020-03-16" AND date <= "2020-11-14" AND stockSymbol="AAPL" ORDER BY date DESC', connection_article_database)

dataFrame

### Pulling Stock Price Data from Database

In [ ]:


data = sqlite_utils._findAllStockPricingForStockSymbol(conn, "AAPL")

print(data)

# Adding Title 
plt.title("Stock Prices")
  
# Labeling the axes 
plt.xlabel("Date") 
plt.ylabel("Closing Price in $")
     

    
xs = [x[0] for x in data]
ys = [y[1] for y in data]


#plt.autoscale(enable=True, axis='both', tight=None)


# Function to plot  
plt.plot(xs, ys)


# function to show the plot  
plt.show() 

# Perform sentiment analysis on the articles